# Thinking Axes for ML-Packages

1. Model Specification or
    * Writing a configuration file
    * Choice of Grammer
        * JSON
            * Caffe
            * Google DistBelief
            * CNTK
        
2. Programmatic generation
    * Writing Code
    * Choice of a High Level Language
        * Lua
            * Torch
        * Python
            * Theano
            * TensorFlow
            * Rich Community
            * Library Infrastructure

# TensorFlow vs Theano

* Theano
    * Deep learning librray
    * Has python wrapper
    * Inspiration for TensorFlow
    * Academic Project
    * Been longer 
    * More stable ?
* TensorFlow
    * Very similar system
    * Better support for distributed systems
    * Google's Project
    * Started slow compared to Theano
    * Pickedup heat after open-sourcing

# What is TensorFlow ?

* Its a deep learning library rite ?
    * Yes, Its a open-source deep learning library
    * Why was it not called DeepFlow ?
        * Its more general
        * Provides primitives for defining functions on Tensors
        * Automatically computing derivatives of arbitrary functions on Tensors
        * One can use TensorFlow to solve PDEs
     * But whats with the Flow ?
         * TensorFlow programs are structured into 2 phases
             1. A Graph Construction Phase
             2. A Graph Execution Phase
    * Graph Construction Phase
         * Assembles a graph, a computation graph
         * Think of Tensors Flowing throught this graph, undergoing transformation at each node
    * Graph Execution Phase
         * Uses a session to execute operations that are specified in this graph.

# Wait.. Whats a Tensor ?

* Formally, 
    * Tensors are multilinear maps from vector spaces to the real numbers
    * Add more formalism.
* Ideas
    * A scalar is a tensor
    * A vector is a tensor
    * A matrix is a tensor
* A Tensor can be represented as
    * A multidimensional array of number
    * So need a N-d array library
    * Why not just use Numpy ?
        * Yes, It has Ndarray Support
        * But cannot create tensor Functions
        * Cannot automatically compute derivatives
        * No GPU support
        * So TensorFlow is a Feature rich N-d Array Library, nothing more
    

# Thinking in Numpy-Land

In [ ]:
import numpy as np

In [ ]:
a = np.zeros((2,2))

In [ ]:
a

In [ ]:
a.shape

In [ ]:
np.reshape(a, (1,4))

In [ ]:
b = np.ones((2,2))

In [ ]:
b

In [ ]:
np.sum(b, axis=1)

# Thinking in TensorFlow-Land

In [ ]:
import tensorflow as tf

In [ ]:
tf.InteractiveSession()

In [ ]:
a = tf.zeros((2,2))

In [ ]:
a

In [ ]:
b = tf.ones((2,2))

In [ ]:
b

In [ ]:
tf.reduce_sum(b, reduction_indices=1).eval()

In [ ]:
a.get_shape()

In [ ]:
tf.reshape(a, (1,4)).eval()

# Numpy to TensorFlow Dictionary

![Dict](np_to_tf_dict.png)

# Explicit Evaluation

In [ ]:
# TensorFlow computations define a computation graph
# This means no numerical value until evaluated explicitly

In [ ]:
a = np.zeros((2,2))

In [ ]:
a

In [ ]:
ta = tf.zeros((2,2))

In [ ]:
print(a)

In [ ]:
print(ta)

In [ ]:
print(ta.eval())

# Session Object

In [ ]:
# A session object encapsulates the environment in which 
# Tensor objects are evaluated

In [ ]:
a = tf.constant(5.0)

In [ ]:
a

In [ ]:
b = tf.constant(6.0)

In [ ]:
b

In [ ]:
c = a * b

In [ ]:
with tf.Session() as session:
    print(session.run(c))
    print(c.eval)

* NOTE-1
    * `tf.InteractiveSession()`
    * syntactic sugar for keeping a default session
    *  open in jupyter or `ipython`

* NOTE-2
    * `session.run(c)`
    * is an example of a tensorFlow Fetch
    * coming up soon

# Computation Graph

* Idea Repeated to stress..
    * Tensorflow program structured as 
        * A Graph construction phase
        * A Graph execution phase
            * This uses a session to execute operations in the graph
    * All computations add nodes to global default graph

# Variables

* All tensors we have used previously have been `constant` tensors
* None of the tensors we have used untill now were variables
* Lets define our first ever tensor variable

In [ ]:
w1 = tf.ones((2,2))
w1

In [ ]:
w2 = tf.Variable(tf.zeros((2,2)), name='weights')
w2

In [ ]:
with tf.Session() as sess:
    # this is a tensor flow constant
    print(sess.run(w1))
    # this is a tensor flow variable
    # please note the init call
    sess.run(tf.global_variables_initializer())
    print(sess.run(w2))

* TensorFlow variables must be initialized before they have values!
* Please contrast with constant tensors.

In [ ]:
W = tf.Variable(tf.zeros((2,2)), name="weights")
# variable objects can be initialized from constants

In [ ]:
R = tf.Variable(tf.random_normal((2,2)), name="random_weights")
# variable objects can be initialized from random values

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(W))
    print(sess.run(R))

# Updating Variable State

In [ ]:
state = tf.Variable(0, name="counter")

In [ ]:
new_value = tf.add(state, tf.constant(1))
# think of this as doing
# new_value = state + 1

In [ ]:
update = tf.assign(state, new_value)
# think of this as state = new_value

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(state))
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))
        
#        
# state = 0
#
# print(state)
# for _ in range(3)
#     state = state + 1
#     print(state)
#


# Fetching Variable State

In [ ]:
input1 = tf.constant(3.0)

In [ ]:
input2 = tf.constant(2.0)

In [ ]:
input3 = tf.constant(5.0)

In [ ]:
intermed = tf.add(input2, input3)

In [ ]:
mul = tf.multiply(input1, intermed)

In [ ]:
with tf.Session() as sess:
    result = sess.run([mul, intermed])
    print(result)
    

# calling sess.run(var) on a tf.Session() object
# retrives its value.

# if you want to retrieve multiple variables
# simultaneously can do
# like sess.run([var1, var2])

![fetch](fetching_variable_state.png)

# Input External Data into TensorFlow

* All Previous examples have manually defined tensors
* How to get external data sets into TensorFlow land ?
* Import from numpy works ...

In [ ]:
a = np.zeros((3,3))

In [ ]:
ta = tf.convert_to_tensor(a)

In [ ]:
with tf.Session() as sess:
    print(sess.run(ta))

# Placeholders

* Getting the data with tf.convert_to_tensor() is cool, but as you 
  see it does not scale.
* Use Dummy nodes that provide entry points for data to the computational graph
    * This takes us to `tf.placeholder` variables
* Now we need a mapping from `tf.placeholder` variables or their names
  to data like numpy arrays, lists, etc.. 
   * This takes us to `feed_dict`
   * This is a python dictionary

In [ ]:
# define placeholder objects for data entry
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

In [ ]:
with tf.Session() as sess:
    print(sess.run([output], feed_dict={input1:[7.], input2:[2.] }))
    
# fetch value of input from computation graph
# feed data into the compuation graph

# Feed Dictionaries

![feed_dict](feed_dict.png)

# Variable Scope

* Complicated tensorFlow models can have 100's of variables
    * `tf.variable_scope()` provides simple name-spacing to avoid clashes
    * `tf.get_variable()` creates/accesses variables from withing a variable scope

* Variable scope is a simple type of namespacing that adds prefixes to variable names within scope


In [ ]:
# with tf.variable_scope("foo"):
#    with tf.variable_scope("bar"):
#        v = tf.get_variable("v", [1])
# assert v.name == "foo/bar/v:0"

* Variable scope control variable reuse

In [ ]:
# with tf.variable_scope("foo"):
#    v = tf.get_variable("v", [1])
#    tf.get_variable_scope().reuse_variables()
#    v1 = tf.get_variable("v", [1])
# assert v1 == v

# Get Variable

* Behaviour of get_variable() depends on
    * reuse is set to false
        * create and return new variable
    * reuse is set to true
        * search for existing variable with given name
        * raise ValueError if none found

In [ ]:
#
# with tf.variable_scope("foo"):
#     v = tf.get_variable("v", [1])
# assert v.name == "foo/v:0"
#

#
# with tf.variable_scope("foo"):
#     v = tf.get_variable("v", [1])
# with tf.variable_scope("foo", reuse=True):
#    v1 = tf.get_variable("v", [1])
# assert v1 = v
#



# Simple TensorFlow Scripts

## TensorFlow Constants

In [ ]:
# this is our first tensorflow line of code
# 1. its a tensorflow constant !
# welcome to tensorflow land
x = tf.constant(35, name='x')

## Build the computation Graph

In [ ]:
y = x + 5

In [ ]:
print(y)

## Run the Computation Graph

In [ ]:
with tf.Session() as session:
    print(session.run(y))

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.image as mpimg

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# which image
filename = "ganesha.jpg"

In [ ]:
# load image
raw_image_data = mpimg.imread(filename)

In [ ]:
# Lord Ganesha was a scribe for Mr. Veda Vyasa
# who was narrating the Mahabharata.
#
# Later today we want to see if GAN's can learn
# the joint distribution over b/w images of Ganesha's
#
# For now, here is how, our god looks like ...
#
# Notice that there are 13 discrete features.

plt.imshow(raw_image_data)

## TensorFlow Variables
#### Stepping into TensorFlow-land from Numpy-land

In [ ]:
# create a
# 1, tenforflow constant (last time)
# 2. tensorflow variable (now)
x = tf.Variable(raw_image_data, name='x')

In [ ]:
# tf.initialize_all_variables() was deprecated recently
model = tf.global_variables_initializer()

In [ ]:
with tf.Session() as session:
    # perform a basic operation
    transpose_op = tf.transpose(x, perm=[1, 0, 2])
    session.run(model)
    result = session.run(transpose_op)

In [ ]:
# he may not like it, but here is the transpose
plt.imshow(result)

## TensorFlow Placeholders

In [ ]:
x = tf.placeholder("float", 3)
# size is optional, but helps

In [ ]:
y = x * 2

In [ ]:
with tf.Session() as session:
    result = session.run(y, feed_dict={x: [1, 2, 3]})
    print(result)

### 2D Placeholder

In [ ]:
x = tf.placeholder("float", [None, 3])
# size can be multidimensional
# None means , you dont know the size now
# Like Data sets used in ML
# You dont want to hardcode the number of samples

In [ ]:
y = x * 2

In [ ]:
x_data = [[1, 2, 3],
          [4, 5, 6],]
#
# this is 2 by 3
# can be  3 by 3
# can be  4 by 3 ...

In [ ]:
with tf.Session() as session:
    result = session.run(y, feed_dict={x: x_data})
    print(result)

### 3D Placeholder

In [ ]:
image = tf.placeholder("uint8", [None, None, 3])

In [ ]:
reverse = tf.reverse(image, [True, False]) # [True, False, False]

In [ ]:
with tf.Session() as session:
    result = session.run(reverse, feed_dict={image: raw_image_data})
    print(result.shape)

In [ ]:
plt.imshow(result)
plt.show()

# Maximum Entropy Classifier (logistic regression)
Minimizing cross-entrpy loss (logistic loss)

# Custom Functions (todo)

# TensorFlow Learn

# Keras